# Chapter 18: LVM (Large Vision-Multimodal Model) Fine-tuning

## 학습 목표
- LVM(Large Vision-Multimodal Model)의 아키텍처와 작동 원리를 이해한다
- Vision Encoder, Projector, LLM의 역할과 상호작용을 파악한다
- 멀티모달 데이터셋 구성 방법을 학습한다
- LLaVA, Qwen-VL 등 주요 LVM 모델을 Fine-tuning한다

---

## 1. LVM (Large Vision-Multimodal Model) 개요

### 1.1 LVM이란?

LVM(Large Vision-Multimodal Model)은 **이미지와 텍스트를 동시에 이해하고 처리**할 수 있는 대규모 멀티모달 모델이다. GPT-4V, Claude 3, Gemini 등이 대표적인 예시이며, 오픈소스로는 LLaVA, Qwen-VL, InternVL 등이 있다.


### 1.2 핵심 구성 요소

| 구성 요소 | 역할 | 예시 |
|----------|------|------|
| **Vision Encoder** | 이미지를 특징 벡터로 변환 | CLIP ViT, SigLIP, EVA |
| **Projector** | 비전 특징을 LLM 임베딩 공간으로 매핑 | MLP, Q-Former, Perceiver |
| **LLM Backbone** | 텍스트 이해 및 생성 | LLaMA, Qwen, Mistral |


## 2. LVM 아키텍처 상세 분석

### 2.1 Vision Encoder

Vision Encoder는 이미지를 고차원 특징 벡터로 변환하는 역할을 한다. 대부분의 LVM은 CLIP이나 SigLIP의 Vision Transformer를 사용한다.

### 2.2 Projector (Vision-Language Connector)

Projector는 Vision Encoder의 출력을 LLM이 이해할 수 있는 형태로 변환한다. 여러 가지 설계 방식이 있다.

| Projector 유형 | 설명 | 사용 모델 |
|---------------|------|----------|
| **Linear** | 단순 선형 변환 | 초기 LLaVA |
| **MLP** | 2-layer MLP | LLaVA-1.5 |
| **Q-Former** | Cross-attention 기반 | BLIP-2, InstructBLIP |
| **Perceiver Resampler** | 가변 길이 → 고정 길이 | Idefics, Flamingo |
| **C-Abstractor** | Convolution 기반 압축 | Honeybee |


### 2.3 멀티모달 입력 처리

LVM은 이미지 토큰과 텍스트 토큰을 결합하여 LLM에 입력한다.

## 3. 환경 설정

In [1]:
# 필요한 라이브러리 설치
%pip install -q pillow requests

Note: you may need to restart the kernel to use updated packages.


In [14]:
import os
import json
import torch
from PIL import Image
import requests
from io import BytesIO
from datasets import load_dataset, Dataset
from transformers import (
    AutoProcessor,
    AutoModelForVision2Seq,
    LlavaForConditionalGeneration,
    LlavaNextForConditionalGeneration,
    Qwen2VLForConditionalGeneration,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

# 경로 설정
BASE_PATH = "./LVM_Finetuning"
OUTPUT_DIR = os.path.join(BASE_PATH, "checkpoints")
os.makedirs(OUTPUT_DIR, exist_ok=True)

PyTorch 버전: 2.6.0+cu126
CUDA 사용 가능: True
GPU: NVIDIA H100 80GB HBM3
VRAM: 79.2 GB


## 4. LVM Fine-tuning 전략

### 4.1 Fine-tuning 단계

LVM Fine-tuning은 보통 2단계로 진행된다.

```
Stage 1: Pre-training (Feature Alignment)
┌─────────────────────────────────────────────────────────────┐
│  목적: Vision Encoder 출력을 LLM 공간에 정렬                      │
│  학습 대상: Projector만 학습                                    │
│  데이터: 이미지-캡션 쌍 (대규모)                                   │
│  예: CC3M, LAION 등                                          │
└─────────────────────────────────────────────────────────────┘
              │
              ▼
Stage 2: Instruction Tuning
┌─────────────────────────────────────────────────────────────┐
│  목적: 지시사항을 따르는 능력 학습                                  │
│  학습 대상: Projector + LLM (선택적)                            │
│  데이터: 멀티모달 대화 데이터                                      │
│  예: LLaVA-Instruct, ShareGPT4V 등                            │
└─────────────────────────────────────────────────────────────┘
```

### 4.2 어떤 부분을 학습할 것인가?

| 전략 | Vision Encoder | Projector | LLM | 특징 |
|------|---------------|-----------|-----|------|
| **Projector Only** | ❄️ Frozen | 🔥 Train | ❄️ Frozen | 빠르고 안정적 |
| **Projector + LLM (LoRA)** | ❄️ Frozen | 🔥 Train | 🔥 LoRA | 권장 방식 |
| **Full Fine-tuning** | 🔥 Train | 🔥 Train | 🔥 Train | 최고 성능, 비용 높음 |

### 4.3 권장 설정

```python
# Vision Encoder: 동결
for param in model.vision_tower.parameters():
    param.requires_grad = False

# Projector: 학습
for param in model.multi_modal_projector.parameters():
    param.requires_grad = True

# LLM: LoRA 적용
lora_config = LoraConfig(
    r=16,
    target_modules=["q_proj", "v_proj"],
)
```

## 5. 멀티모달 데이터셋 구성

### 5.1 데이터 형식

LVM 학습 데이터는 보통 다음과 같은 형식을 따른다.

```json
{
    "id": "unique_id",
    "image": "path/to/image.jpg",
    "conversations": [
        {"from": "human", "value": "<image>\n이 이미지를 설명해줘."},
        {"from": "gpt", "value": "이 이미지에는 푸른 하늘 아래..."}
    ]
}
```

In [15]:
# 샘플 멀티모달 데이터셋 생성
def create_sample_multimodal_dataset():
    """
    샘플 멀티모달 데이터셋을 생성하는 함수다.
    실제 사용 시에는 실제 이미지 경로와 대화를 사용한다.
    """
    samples = [
        {
            "id": "sample_001",
            "image": "https://upload.wikimedia.org/wikipedia/commons/thumb/4/47/PNG_transparency_demonstration_1.png/300px-PNG_transparency_demonstration_1.png",
            "conversations": [
                {"from": "human", "value": "<image>\n이 이미지에서 무엇이 보이나?"},
                {"from": "gpt", "value": "이 이미지에는 주사위 두 개가 보인다. 투명한 배경 위에 놓여 있으며, 각 주사위의 면에는 점이 표시되어 있다."}
            ]
        },
        {
            "id": "sample_002",
            "image": "https://upload.wikimedia.org/wikipedia/commons/thumb/b/b6/Image_created_with_a_mobile_phone.png/1200px-Image_created_with_a_mobile_phone.png",
            "conversations": [
                {"from": "human", "value": "<image>\n이 사진의 분위기를 설명해줘."},
                {"from": "gpt", "value": "이 사진은 따뜻하고 평화로운 분위기를 자아낸다. 황금빛 햇살이 풍경을 비추고 있으며, 자연의 아름다움이 느껴진다."}
            ]
        },
        {
            "id": "sample_003",
            "image": "https://upload.wikimedia.org/wikipedia/commons/4/41/Siberischer_tiger_de_edit02.jpg",
            "conversations": [
                {"from": "human", "value": "<image>\n이 동물에 대해 알려줘."},
                {"from": "gpt", "value": "이것은 시베리아 호랑이로, 세계에서 가장 큰 고양이과 동물이다. 주황색 털에 검은 줄무늬가 특징이며, 멸종 위기에 처해 있다."}
            ]
        }
    ]
    
    return samples

sample_data = create_sample_multimodal_dataset()
print(f"샘플 데이터셋 생성 완료: {len(sample_data)}개")
print(f"\n첫 번째 샘플:")
print(json.dumps(sample_data[0], indent=2, ensure_ascii=False))

샘플 데이터셋 생성 완료: 3개

첫 번째 샘플:
{
  "id": "sample_001",
  "image": "https://upload.wikimedia.org/wikipedia/commons/thumb/4/47/PNG_transparency_demonstration_1.png/300px-PNG_transparency_demonstration_1.png",
  "conversations": [
    {
      "from": "human",
      "value": "<image>\n이 이미지에서 무엇이 보이나?"
    },
    {
      "from": "gpt",
      "value": "이 이미지에는 주사위 두 개가 보인다. 투명한 배경 위에 놓여 있으며, 각 주사위의 면에는 점이 표시되어 있다."
    }
  ]
}


In [16]:
# 이미지 로드 유틸리티
def load_image(image_path_or_url: str, timeout: int = 20) -> Image.Image:
    """
    로컬 경로 또는 URL에서 이미지를 로드하는 함수다.
    - URL이 이미지가 아닌 응답(HTML/JSON/403 등)을 주면 원인 파악 가능한 에러를 던진다.
    """
    if image_path_or_url.startswith("http"):
        headers = {
            "User-Agent": "Mozilla/5.0",
            "Accept": "image/avif,image/webp,image/apng,image/*,*/*;q=0.8",
        }
        r = requests.get(
            image_path_or_url,
            headers=headers,
            timeout=timeout,
            allow_redirects=True,
        )
        r.raise_for_status()

        ctype = (r.headers.get("Content-Type") or "").lower()
        if not ctype.startswith("image/"):
            snippet = (r.text[:300] if r.text else "").replace("\n", " ")
            raise ValueError(
                f"URL이 이미지를 반환하지 않음. Content-Type={ctype}, body[:300]={snippet}"
            )

        try:
            return Image.open(BytesIO(r.content)).convert("RGB")
        except Exception as e:
            raise ValueError(f"이미지 디코딩 실패: {e} (Content-Type={ctype})") from e

    # local path
    return Image.open(image_path_or_url).convert("RGB")


# 테스트
test_image = load_image(sample_data[0]["image"])
print(f"이미지 로드 성공: {test_image.size}")

이미지 로드 성공: (300, 225)


### 5.2 공개 멀티모달 데이터셋

In [17]:
# 공개 멀티모달 데이터셋 목록
public_datasets = {
    # 이미지-텍스트 쌍
    "image_caption": [
        "HuggingFaceM4/COCO",
        "google-research-datasets/conceptual_captions",
    ],
    
    # VQA (Visual Question Answering)
    "vqa": [
        "HuggingFaceM4/VQAv2",
        "textvqa/textvqa",
    ],
    
    # 멀티모달 대화
    "conversation": [
        "liuhaotian/LLaVA-Instruct-150K",
        "Lin-Chen/ShareGPT4V",
    ],
    
    # 문서 이해
    "document": [
        "HuggingFaceM4/DocumentVQA",
        "naver-clova-ix/synthdog-en",
    ]
}

print("공개 멀티모달 데이터셋:")
for category, datasets in public_datasets.items():
    print(f"\n[{category}]")
    for ds in datasets:
        print(f"  - {ds}")

공개 멀티모달 데이터셋:

[image_caption]
  - HuggingFaceM4/COCO
  - google-research-datasets/conceptual_captions

[vqa]
  - HuggingFaceM4/VQAv2
  - textvqa/textvqa

[conversation]
  - liuhaotian/LLaVA-Instruct-150K
  - Lin-Chen/ShareGPT4V

[document]
  - HuggingFaceM4/DocumentVQA
  - naver-clova-ix/synthdog-en


In [18]:
# COCO 캡션 데이터셋 로드 예시
def load_coco_sample():
    """
    COCO 캡션 데이터셋 샘플을 로드하는 함수다.
    """
    try:
        dataset = load_dataset("HuggingFaceM4/COCO", split="train[:100]")
        print(f"COCO 데이터셋 로드 완료: {len(dataset)}개")
        print(f"컬럼: {dataset.column_names}")
        return dataset
    except Exception as e:
        print(f"데이터셋 로드 실패: {e}")
        return None

# coco_dataset = load_coco_sample()
print("COCO 데이터셋 로드 함수 준비 완료")

COCO 데이터셋 로드 함수 준비 완료


## 6. LLaVA 모델 Fine-tuning

### 6.1 LLaVA (Large Language and Vision Assistant)

LLaVA는 가장 널리 사용되는 오픈소스 LVM 중 하나로, 간단하면서도 효과적인 구조를 가지고 있다.

In [26]:
# LLaVA 모델 설정
LLAVA_MODELS = {
    "llava-1.5-7b": "llava-hf/llava-1.5-7b-hf",
    "llava-1.5-13b": "llava-hf/llava-1.5-13b-hf",
    "llava-v1.6-mistral-7b": "llava-hf/llava-v1.6-mistral-7b-hf",
    "llava-v1.6-vicuna-7b": "llava-hf/llava-v1.6-vicuna-7b-hf",
}

# 테스트용으로 작은 모델 선택
MODEL_ID = "llava-hf/llava-1.5-7b-hf"

print("사용 가능한 LLaVA 모델:")
for name, model_id in LLAVA_MODELS.items():
    print(f"  - {name}: {model_id}")

사용 가능한 LLaVA 모델:
  - llava-1.5-7b: llava-hf/llava-1.5-7b-hf
  - llava-1.5-13b: llava-hf/llava-1.5-13b-hf
  - llava-v1.6-mistral-7b: llava-hf/llava-v1.6-mistral-7b-hf
  - llava-v1.6-vicuna-7b: llava-hf/llava-v1.6-vicuna-7b-hf


In [27]:
# 4-bit 양자화 설정 (메모리 절약)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

print("4-bit 양자화 설정 완료")

4-bit 양자화 설정 완료


In [33]:
def load_llava_model(model_id, use_4bit=False):  # use_4bit=False로 변경
    """
    LLaVA 모델과 프로세서를 로드하는 함수다.
    """
    print(f"LLaVA 모델 로딩: {model_id}")
    
    processor = AutoProcessor.from_pretrained(model_id)
    
    model = LlavaForConditionalGeneration.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
        # 양자화 없이 로드
    )
    
    print(f"모델 로드 완료: {model.num_parameters():,} 파라미터")
    
    return model, processor

# 로드 (실제 실행 시 주석 해제)
# model, processor = load_llava_model(MODEL_ID, use_4bit=True)
print("LLaVA 로드 함수 준비 완료")

LLaVA 로드 함수 준비 완료


### 6.2 LLaVA용 LoRA 설정

In [34]:
def get_llava_lora_config():
    """
    LLaVA 모델에 최적화된 LoRA 설정을 반환하는 함수다.
    """
    lora_config = LoraConfig(
        r=16,
        lora_alpha=32,
        target_modules=[
            "q_proj", "k_proj", "v_proj", "o_proj",  # Attention
            "gate_proj", "up_proj", "down_proj",     # MLP
        ],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
    )
    return lora_config

llava_lora_config = get_llava_lora_config()
print("LLaVA LoRA 설정:")
print(f"  r: {llava_lora_config.r}")
print(f"  alpha: {llava_lora_config.lora_alpha}")
print(f"  target_modules: {llava_lora_config.target_modules}")
print(f"  modules_to_save: {llava_lora_config.modules_to_save}")

LLaVA LoRA 설정:
  r: 16
  alpha: 32
  target_modules: {'o_proj', 'v_proj', 'k_proj', 'gate_proj', 'q_proj', 'down_proj', 'up_proj'}
  modules_to_save: None


In [35]:
def prepare_llava_for_training(model):
    """
    LLaVA 모델을 Fine-tuning용으로 준비하는 함수다.
    """
    # 1. Vision Encoder 동결
    for param in model.vision_tower.parameters():
        param.requires_grad = False
    print("✓ Vision Encoder 동결 완료")
    
    # 2. Projector 학습 가능 설정
    for param in model.multi_modal_projector.parameters():
        param.requires_grad = True
    print("✓ Projector 학습 가능 설정 완료")
    
    # 3. LoRA 적용
    lora_config = LoraConfig(
        r=16,
        lora_alpha=32,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        modules_to_save=["multi_modal_projector"],  # 양자화 없을 때만 사용 가능
    )
    model = get_peft_model(model, lora_config)
    print("✓ LoRA 적용 완료")
    
    model.print_trainable_parameters()
    
    return model

print("LLaVA 학습 준비 함수 준비 완료")

LLaVA 학습 준비 함수 준비 완료


### 6.3 데이터 전처리

In [36]:
def preprocess_llava_data(examples, processor, max_length=2048):
    """
    LLaVA 학습 데이터를 전처리하는 함수다.
    """
    images = []
    texts = []
    
    for example in examples:
        # 이미지 로드
        image = load_image(example["image"])
        images.append(image)
        
        # 대화 형식 구성
        conversation = example["conversations"]
        
        # LLaVA 형식으로 변환
        text = ""
        for turn in conversation:
            if turn["from"] == "human":
                text += f"USER: {turn['value']}\n"
            else:
                text += f"ASSISTANT: {turn['value']}\n"
        
        texts.append(text)
    
    # 프로세서로 배치 처리
    batch = processor(
        text=texts,
        images=images,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    
    # labels 설정
    batch["labels"] = batch["input_ids"].clone()
    
    return batch

print("데이터 전처리 함수 준비 완료")

데이터 전처리 함수 준비 완료


### 6.4 LLaVA Fine-tuning 실행

In [37]:
def finetune_llava(
    model_id,
    train_data,
    output_dir,
    num_epochs=3,
    learning_rate=2e-5,
    batch_size=2,
):
    """
    LLaVA 모델을 Fine-tuning하는 함수다.
    """
    print("=" * 60)
    print("LLaVA Fine-tuning 시작")
    print("=" * 60)
    
    # 1. 모델 로드
    print("\n[1/4] 모델 로딩...")
    model, processor = load_llava_model(model_id, use_4bit=True)
    
    # 2. 학습 준비
    print("\n[2/4] 학습 준비...")
    model = prepare_llava_for_training(model)
    
    # 3. 데이터 전처리
    print("\n[3/4] 데이터 전처리...")
    train_dataset = preprocess_llava_data(train_data, processor)
    
    # 4. 학습 설정
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=num_epochs,
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=4,
        learning_rate=learning_rate,
        warmup_ratio=0.1,
        lr_scheduler_type="cosine",
        bf16=True,
        logging_steps=10,
        save_strategy="epoch",
        remove_unused_columns=False,
        report_to="none",
    )
    
    # 5. Trainer 생성 및 학습
    print("\n[4/4] 학습 시작...")
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
    )
    
    trainer.train()
    
    # 저장
    trainer.save_model(output_dir)
    processor.save_pretrained(output_dir)
    
    print("\n" + "=" * 60)
    print("LLaVA Fine-tuning 완료!")
    print(f"모델 저장 위치: {output_dir}")
    print("=" * 60)
    
    return model, processor

# Train
model, processor = finetune_llava(
    model_id=MODEL_ID,
    train_data=sample_data,
    output_dir=os.path.join(OUTPUT_DIR, "LLAVA_Finetuned"),
)

print("LLaVA Fine-tuning 함수 준비 완료")

LLaVA Fine-tuning 시작

[1/4] 모델 로딩...
LLaVA 모델 로딩: llava-hf/llava-1.5-7b-hf


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

모델 로드 완료: 7,063,427,072 파라미터

[2/4] 학습 준비...
✓ Vision Encoder 동결 완료
✓ Projector 학습 가능 설정 완료
✓ LoRA 적용 완료
trainable params: 40,116,224 || all params: 7,103,543,296 || trainable%: 0.5647

[3/4] 데이터 전처리...


HTTPError: 429 Client Error: Too many requests. Please contact noc@wikimedia.org for further information (0068e25) for url: https://upload.wikimedia.org/wikipedia/commons/thumb/b/b6/Image_created_with_a_mobile_phone.png/1200px-Image_created_with_a_mobile_phone.png

## 7. Qwen2-VL Fine-tuning

### 7.1 Qwen2-VL 소개

Qwen2-VL은 Alibaba에서 개발한 최신 LVM으로, 동적 해상도와 동영상 이해를 지원한다.

In [49]:
# Qwen2-VL 모델 설정
QWEN2_VL_MODELS = {
    "qwen2-vl-2b": "Qwen/Qwen2-VL-2B-Instruct",
    "qwen2-vl-7b": "Qwen/Qwen2-VL-7B-Instruct",
    "qwen2-vl-72b": "Qwen/Qwen2-VL-72B-Instruct",
}

print("사용 가능한 Qwen2-VL 모델:")
for name, model_id in QWEN2_VL_MODELS.items():
    print(f"  - {name}: {model_id}")

사용 가능한 Qwen2-VL 모델:
  - qwen2-vl-2b: Qwen/Qwen2-VL-2B-Instruct
  - qwen2-vl-7b: Qwen/Qwen2-VL-7B-Instruct
  - qwen2-vl-72b: Qwen/Qwen2-VL-72B-Instruct


In [50]:
def load_qwen2_vl_model(model_id, use_4bit=True):
    """
    Qwen2-VL 모델과 프로세서를 로드하는 함수다.
    """
    print(f"Qwen2-VL 모델 로딩: {model_id}")
    
    # 프로세서 로드
    processor = AutoProcessor.from_pretrained(model_id)
    
    # 모델 로드
    model_kwargs = {
        "torch_dtype": torch.bfloat16,
        "device_map": "auto",
        "trust_remote_code": True,
    }
    
    if use_4bit:
        model_kwargs["quantization_config"] = bnb_config
    
    model = Qwen2VLForConditionalGeneration.from_pretrained(
        model_id,
        **model_kwargs
    )
    
    print(f"모델 로드 완료: {model.num_parameters():,} 파라미터")
    
    return model, processor

print("Qwen2-VL 로드 함수 준비 완료")

Qwen2-VL 로드 함수 준비 완료


In [51]:
def get_qwen2_vl_lora_config():
    """
    Qwen2-VL 모델에 최적화된 LoRA 설정을 반환하는 함수다.
    """
    lora_config = LoraConfig(
        r=16,
        lora_alpha=32,
        target_modules=[
            "q_proj", "k_proj", "v_proj", "o_proj",
            "gate_proj", "up_proj", "down_proj",
        ],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
    )
    return lora_config

print("Qwen2-VL LoRA 설정 함수 준비 완료")

Qwen2-VL LoRA 설정 함수 준비 완료


In [52]:
def preprocess_qwen2_vl_data(examples, processor, max_length=2048):
    """
    Qwen2-VL 학습 데이터를 전처리하는 함수다.
    """
    processed_data = []
    
    for example in examples:
        # 이미지 로드
        image = load_image(example["image"])
        
        # Qwen2-VL 형식으로 메시지 구성
        messages = []
        for turn in example["conversations"]:
            if turn["from"] == "human":
                # 이미지가 포함된 첫 번째 사용자 메시지
                if "<image>" in turn["value"]:
                    content = [
                        {"type": "image", "image": image},
                        {"type": "text", "text": turn["value"].replace("<image>\n", "")}
                    ]
                else:
                    content = [{"type": "text", "text": turn["value"]}]
                messages.append({"role": "user", "content": content})
            else:
                messages.append({"role": "assistant", "content": turn["value"]})
        
        # 프로세서로 처리
        text = processor.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )
        
        processed_data.append({
            "text": text,
            "images": [image]
        })
    
    return processed_data

print("Qwen2-VL 데이터 전처리 함수 준비 완료")

Qwen2-VL 데이터 전처리 함수 준비 완료


## 8. 모델 추론 테스트

In [53]:
def test_llava_inference(model, processor, image_path, question):
    """
    LLaVA 모델로 추론을 수행하는 함수다.
    """
    # 이미지 로드
    image = load_image(image_path)
    
    # 프롬프트 구성
    prompt = f"USER: <image>\n{question}\nASSISTANT:"
    
    # 입력 처리
    inputs = processor(
        text=prompt,
        images=image,
        return_tensors="pt"
    ).to(model.device)
    
    # 생성
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            do_sample=True,
        )
    
    # 디코딩
    response = processor.decode(outputs[0], skip_special_tokens=True)
    
    # ASSISTANT: 이후 부분만 추출
    answer = response.split("ASSISTANT:")[-1].strip()
    
    return answer

print("LLaVA 추론 함수 준비 완료")

LLaVA 추론 함수 준비 완료


In [54]:
def test_qwen2_vl_inference(model, processor, image_path, question):
    """
    Qwen2-VL 모델로 추론을 수행하는 함수다.
    """
    # 이미지 로드
    image = load_image(image_path)
    
    # 메시지 구성
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image},
                {"type": "text", "text": question}
            ]
        }
    ]
    
    # 프롬프트 생성
    text = processor.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    # 입력 처리
    inputs = processor(
        text=[text],
        images=[image],
        return_tensors="pt",
        padding=True
    ).to(model.device)
    
    # 생성
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            do_sample=True,
        )
    
    # 디코딩
    response = processor.batch_decode(
        outputs[:, inputs.input_ids.shape[1]:],
        skip_special_tokens=True
    )[0]
    
    return response

print("Qwen2-VL 추론 함수 준비 완료")

Qwen2-VL 추론 함수 준비 완료


## 9. LVM Fine-tuning 베스트 프랙티스

### **LVM 파인튜닝 핵심 가이드**

**1. 학습 전략 (Training Strategy)**
모델의 각 구성 요소별로 학습 여부를 다르게 설정하여 효율성을 극대화합니다.

* **Vision Encoder:** 대부분 **동결(Freeze)**하여 학습하지 않음.
* **Projector:** 시각 정보와 언어 모델을 연결하는 핵심이므로 **필수 학습**.
* **LLM:** 전체 파라미터 학습보다는 **LoRA(Low-Rank Adaptation)** 적용을 권장.

**2. 메모리 최적화 (Memory Optimization)**
거대 모델을 효율적으로 학습하기 위한 필수 설정입니다.

* **양자화:** 4-bit 양자화(QLoRA)를 사용하여 VRAM 사용량 절감.
* **체크포인팅:** Gradient Checkpointing을 활성화하여 메모리 효율 확보.
* **배치 설정:** 작은 배치 사이즈(Batch size)를 사용하되, **Gradient Accumulation(그래디언트 누적)**을 통해 큰 배치 효과를 냄.

**3. 학습률 설정 (Learning Rate)**
학습 대상에 따라 학습률(LR)을 다르게 가져가는 것이 중요합니다.

| 학습 대상 | 권장 학습률 (Learning Rate) | 비고 |
| --- | --- | --- |
| **Projector** | `1e-4` ~ `2e-4` | 비교적 높게 설정 |
| **LLM (LoRA)** | `1e-5` ~ `2e-5` | 미세 조정이므로 낮게 설정 |
| **통합 학습** | `2e-5` | Projector와 LLM 동시 학습 시 |

**4. 데이터 품질 (Data Quality)**
모델 성능에 가장 큰 영향을 미치는 요소입니다.

* **고품질 데이터:** 노이즈가 없는 명확한 이미지-텍스트 쌍 사용.
* **해상도 다양화:** 다양한 크기와 비율의 이미지를 포함하여 일반화 성능 향상.
* **상세한 설명:** 단순한 태깅보다는 상세하고 정확한 캡션/대화 데이터 권장.

**5. 평가 지표 (Evaluation)**
다양한 관점에서 모델의 성능을 검증합니다.

* **정량적 평가 (캡셔닝):** BLEU, CIDEr, METEOR 등 텍스트 유사도 지표.
* **과제별 평가 (VQA):** 질문에 대한 정답률(Accuracy).
* **정성적 평가:** GPT-4V를 이용한 자동 평가 또는 사람에 의한 직접 평가.

## 10. 도메인 특화 LVM Fine-tuning 예시

### 10.1 의료 이미지 분석

In [55]:
# 의료 이미지 분석용 데이터 형식 예시
medical_data_example = {
    "id": "medical_001",
    "image": "path/to/xray_image.jpg",
    "conversations": [
        {
            "from": "human",
            "value": "<image>\n이 흉부 X-ray 이미지에서 이상 소견이 있는가?"
        },
        {
            "from": "gpt",
            "value": "이 흉부 X-ray 이미지를 분석한 결과, 좌측 폐 하엽에 경화(consolidation) 소견이 관찰된다. 이는 폐렴을 시사할 수 있으나, 정확한 진단을 위해서는 임상 증상과 함께 추가적인 검사가 필요하다. 반드시 전문 의료진과 상담할 것을 권고한다."
        }
    ]
}

print("[의료 이미지 분석 데이터 형식]")
print(json.dumps(medical_data_example, indent=2, ensure_ascii=False))

[의료 이미지 분석 데이터 형식]
{
  "id": "medical_001",
  "image": "path/to/xray_image.jpg",
  "conversations": [
    {
      "from": "human",
      "value": "<image>\n이 흉부 X-ray 이미지에서 이상 소견이 있는가?"
    },
    {
      "from": "gpt",
      "value": "이 흉부 X-ray 이미지를 분석한 결과, 좌측 폐 하엽에 경화(consolidation) 소견이 관찰된다. 이는 폐렴을 시사할 수 있으나, 정확한 진단을 위해서는 임상 증상과 함께 추가적인 검사가 필요하다. 반드시 전문 의료진과 상담할 것을 권고한다."
    }
  ]
}


### 10.2 문서 이해 (OCR + 이해)

In [56]:
# 문서 이해용 데이터 형식 예시
document_data_example = {
    "id": "document_001",
    "image": "path/to/invoice.jpg",
    "conversations": [
        {
            "from": "human",
            "value": "<image>\n이 청구서에서 총 금액은 얼마인가?"
        },
        {
            "from": "gpt",
            "value": "이 청구서의 총 금액은 1,250,000원이다. 품목별로 보면 제품 A가 500,000원, 제품 B가 750,000원이며, 부가세 포함 금액이다."
        }
    ]
}

print("[문서 이해 데이터 형식]")
print(json.dumps(document_data_example, indent=2, ensure_ascii=False))

[문서 이해 데이터 형식]
{
  "id": "document_001",
  "image": "path/to/invoice.jpg",
  "conversations": [
    {
      "from": "human",
      "value": "<image>\n이 청구서에서 총 금액은 얼마인가?"
    },
    {
      "from": "gpt",
      "value": "이 청구서의 총 금액은 1,250,000원이다. 품목별로 보면 제품 A가 500,000원, 제품 B가 750,000원이며, 부가세 포함 금액이다."
    }
  ]
}


### 10.3 제품 이미지 분석

In [57]:
# 제품 이미지 분석용 데이터 형식 예시
product_data_example = {
    "id": "product_001",
    "image": "path/to/product.jpg",
    "conversations": [
        {
            "from": "human",
            "value": "<image>\n이 제품의 특징을 분석해줘."
        },
        {
            "from": "gpt",
            "value": "이 제품은 무선 블루투스 이어폰으로, 다음과 같은 특징이 있다:\n\n1. 디자인: 화이트 컬러의 인이어 타입\n2. 충전 케이스: 컴팩트한 원형 케이스 포함\n3. 예상 착용감: 인체공학적 설계\n4. 브랜드: 이미지에서 확인되는 로고 기반으로 프리미엄 브랜드로 추정\n\n정확한 스펙은 제품 상세 페이지를 확인하는 것이 좋다."
        }
    ]
}

print("[제품 이미지 분석 데이터 형식]")
print(json.dumps(product_data_example, indent=2, ensure_ascii=False))

[제품 이미지 분석 데이터 형식]
{
  "id": "product_001",
  "image": "path/to/product.jpg",
  "conversations": [
    {
      "from": "human",
      "value": "<image>\n이 제품의 특징을 분석해줘."
    },
    {
      "from": "gpt",
      "value": "이 제품은 무선 블루투스 이어폰으로, 다음과 같은 특징이 있다:\n\n1. 디자인: 화이트 컬러의 인이어 타입\n2. 충전 케이스: 컴팩트한 원형 케이스 포함\n3. 예상 착용감: 인체공학적 설계\n4. 브랜드: 이미지에서 확인되는 로고 기반으로 프리미엄 브랜드로 추정\n\n정확한 스펙은 제품 상세 페이지를 확인하는 것이 좋다."
    }
  ]
}


## 11. 동영상 이해 모델 (Video LLM)

In [58]:
# 동영상 LLM 모델 목록
video_llm_models = {
    "Qwen2-VL": {
        "model_id": "Qwen/Qwen2-VL-7B-Instruct",
        "특징": "이미지 + 동영상 통합 지원",
        "max_frames": "동적 샘플링"
    },
    "LLaVA-NeXT-Video": {
        "model_id": "llava-hf/LLaVA-NeXT-Video-7B-hf",
        "특징": "비디오 특화",
        "max_frames": 32
    },
    "Video-LLaVA": {
        "model_id": "LanguageBind/Video-LLaVA-7B-hf",
        "특징": "이미지 + 비디오 통합 인코더",
        "max_frames": 8
    },
    "InternVL2-Video": {
        "model_id": "OpenGVLab/InternVL2-8B",
        "특징": "고성능 비디오 이해",
        "max_frames": 64
    }
}

print("동영상 이해 LLM 모델:")
print("=" * 60)
for name, info in video_llm_models.items():
    print(f"\n[{name}]")
    print(f"  Model: {info['model_id']}")
    print(f"  특징: {info['특징']}")
    print(f"  Max Frames: {info['max_frames']}")

동영상 이해 LLM 모델:

[Qwen2-VL]
  Model: Qwen/Qwen2-VL-7B-Instruct
  특징: 이미지 + 동영상 통합 지원
  Max Frames: 동적 샘플링

[LLaVA-NeXT-Video]
  Model: llava-hf/LLaVA-NeXT-Video-7B-hf
  특징: 비디오 특화
  Max Frames: 32

[Video-LLaVA]
  Model: LanguageBind/Video-LLaVA-7B-hf
  특징: 이미지 + 비디오 통합 인코더
  Max Frames: 8

[InternVL2-Video]
  Model: OpenGVLab/InternVL2-8B
  특징: 고성능 비디오 이해
  Max Frames: 64


In [59]:
def process_video_frames(video_path, num_frames=8):
    """
    동영상에서 프레임을 추출하는 함수다.
    """
    try:
        import cv2
        
        cap = cv2.VideoCapture(video_path)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        # 균등하게 프레임 샘플링
        frame_indices = [int(i * total_frames / num_frames) for i in range(num_frames)]
        
        frames = []
        for idx in frame_indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
            ret, frame = cap.read()
            if ret:
                # BGR to RGB
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frames.append(Image.fromarray(frame))
        
        cap.release()
        return frames
        
    except ImportError:
        print("cv2가 설치되지 않았다. pip install opencv-python을 실행한다.")
        return None

print("동영상 프레임 추출 함수 준비 완료")

동영상 프레임 추출 함수 준비 완료


## 12. 요약

### LVM Fine-tuning 핵심 포인트

| 항목 | 설명 |
|------|------|
| **구조** | Vision Encoder + Projector + LLM |
| **권장 전략** | Vision Encoder 동결, Projector 학습, LLM LoRA |
| **데이터 형식** | 이미지 + 대화 쌍 (JSON) |
| **학습률** | Projector: 1e-4, LLM: 1e-5 |
| **메모리 최적화** | 4-bit 양자화, Gradient Checkpointing |

### 주요 오픈소스 모델

| 모델 | 특징 | 추천 용도 |
|------|------|----------|
| **LLaVA** | 간단, 널리 사용 | 일반 목적 |
| **Qwen2-VL** | 동적 해상도, 동영상 | 고품질 요구 |
| **InternVL** | 대규모 비전 인코더 | 세밀한 이해 |
| **Phi-3-Vision** | 경량 | 엣지 배포 |

---

이 튜토리얼을 통해 LVM(Large Vision-Multimodal Model)의 구조를 이해하고, LLaVA, Qwen2-VL 등 주요 모델을 Fine-tuning하는 방법을 학습했다.